In [2]:
import numpy as np
import pandas as pd
from matplotlib.pyplot import subplots
import statsmodels.api as sm
from ISLP import load_data
from ISLP.models import (ModelSpec as MS, summarize)

# default library imports

In [3]:
from ISLP import confusion_table
from ISLP.models import contrast
from sklearn.discriminant_analysis import (LinearDiscriminantAnalysis as LDA, QuadraticDiscriminantAnalysis as QDA)
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [4]:
Smarket = load_data("Smarket")
Smarket

,Year,Lag1,Lag2,Lag3,Lag4,Lag5,Volume,Today,Direction
0,2001,0.381,-0.192,-2.624,-1.055,5.010,1.19130,0.959,Up
1,2001,0.959,0.381,-0.192,-2.624,-1.055,1.29650,1.032,Up
2,2001,1.032,0.959,0.381,-0.192,-2.624,1.41120,-0.623,Down
3,2001,-0.623,1.032,0.959,0.381,-0.192,1.27600,0.614,Up
4,2001,0.614,-0.623,1.032,0.959,0.381,1.20570,0.213,Up
...,...,...,...,...,...,...,...,...,...
1245,2005,0.422,0.252,-0.024,-0.584,-0.285,1.88850,0.043,Up
1246,2005,0.043,0.422,0.252,-0.024,-0.584,1.28581,-0.955,Down
1247,2005,-0.955,0.043,0.422,0.252,-0.024,1.54047,0.130,Up
1248,2005,0.130,-0.955,0.043,0.422,0.252,1.42236,-0.298,Down


In [5]:
Smarket.columns

Index(['Year', 'Lag1', 'Lag2', 'Lag3', 'Lag4', 'Lag5', 'Volume', 'Today',
       'Direction'],
      dtype='object')

In [6]:
Smarket.corr()

ValueError: could not convert string to float: 'Up'

- Here the corr will return an error because there's a qualitative predictor - Direction.
- By that, I attempted to drop the "Direction" column and get the correlation matrix.

In [7]:
df = Smarket.drop(columns=["Direction"])
df.corr()

,Year,Lag1,Lag2,Lag3,Lag4,Lag5,Volume,Today
Year,1.000000,0.029700,0.030596,0.033195,0.035689,0.029788,0.539006,0.030095
Lag1,0.029700,1.000000,-0.026294,-0.010803,-0.002986,-0.005675,0.040910,-0.026155
Lag2,0.030596,-0.026294,1.000000,-0.025897,-0.010854,-0.003558,-0.043383,-0.010250
Lag3,0.033195,-0.010803,-0.025897,1.000000,-0.024051,-0.018808,-0.041824,-0.002448
Lag4,0.035689,-0.002986,-0.010854,-0.024051,1.000000,-0.027084,-0.048414,-0.006900
Lag5,0.029788,-0.005675,-0.003558,-0.018808,-0.027084,1.000000,-0.022002,-0.034860
Volume,0.539006,0.040910,-0.043383,-0.041824,-0.048414,-0.022002,1.000000,0.014592
Today,0.030095,-0.026155,-0.010250,-0.002448,-0.006900,-0.034860,0.014592,1.000000


In [8]:
# Build the model 

allvars = Smarket.drop(columns=["Year" , "Today" , "Direction"])
X = MS(allvars).fit_transform(Smarket)
y = Smarket["Direction"] == "Up"
glm = sm.GLM(y , X , family = sm.families.Binomial())
result = glm.fit()
summarize(result)

,coef,std err,z,P>|z|
intercept,-0.1260,0.241,-0.523,0.601
Lag1,-0.0731,0.050,-1.457,0.145
Lag2,-0.0423,0.050,-0.845,0.398
Lag3,0.0111,0.050,0.222,0.824
Lag4,0.0094,0.050,0.187,0.851
Lag5,0.0103,0.050,0.208,0.835
Volume,0.1354,0.158,0.855,0.392


In [9]:
result.params

intercept   -0.126000
Lag1        -0.073074
Lag2        -0.042301
Lag3         0.011085
Lag4         0.009359
Lag5         0.010313
Volume       0.135441
dtype: float64

In [10]:
result.pvalues

intercept    0.600700
Lag1         0.145232
Lag2         0.398352
Lag3         0.824334
Lag4         0.851445
Lag5         0.834998
Volume       0.392404
dtype: float64

In [11]:
probs = result.predict()
probs[:10]

array([0.50708413, 0.48146788, 0.48113883, 0.51522236, 0.51078116,
       0.50695646, 0.49265087, 0.50922916, 0.51761353, 0.48883778])

In [12]:
labels = np.array(['Down'] * len(Smarket['Lag1']))
labels[probs > 0.5] = "Up"
labels

array(['Up', 'Down', 'Down', ..., 'Up', 'Up', 'Up'],
      shape=(1250,), dtype='<U4')

In [13]:
confusion_table(labels , Smarket['Direction'])

Truth,Down,Up
Predicted,,
Down,145,141
Up,457,507


In [14]:
(145 + 507) / 1250 , np.mean(labels == Smarket['Direction'])

(0.5216, np.float64(0.5216))

In [15]:
train = (Smarket.Year < 2005)
Smarket_train = Smarket.loc[train]
Smarket_test = Smarket.loc[~train]
Smarket_test

,Year,Lag1,Lag2,Lag3,Lag4,Lag5,Volume,Today,Direction
998,2005,-0.134,0.008,-0.007,0.715,-0.431,0.78690,-0.812,Down
999,2005,-0.812,-0.134,0.008,-0.007,0.715,1.51080,-1.167,Down
1000,2005,-1.167,-0.812,-0.134,0.008,-0.007,1.72100,-0.363,Down
1001,2005,-0.363,-1.167,-0.812,-0.134,0.008,1.73890,0.351,Up
1002,2005,0.351,-0.363,-1.167,-0.812,-0.134,1.56910,-0.143,Down
...,...,...,...,...,...,...,...,...,...
1245,2005,0.422,0.252,-0.024,-0.584,-0.285,1.88850,0.043,Up
1246,2005,0.043,0.422,0.252,-0.024,-0.584,1.28581,-0.955,Down
1247,2005,-0.955,0.043,0.422,0.252,-0.024,1.54047,0.130,Up
1248,2005,0.130,-0.955,0.043,0.422,0.252,1.42236,-0.298,Down


In [16]:
X_train = X.loc[train]
y_train = y.loc[train]
X_test = X.loc[~train]
y_test = y.loc[~train]

glm_train = sm.GLM(y_train , X_train , family = sm.families.Binomial())
result = glm_train.fit()
summarize(result)
probs = result.predict(exog=X_test)

In [17]:
D = Smarket.Direction
D_train , D_test = D.loc[train] , D.loc[~train]

In [18]:
labels = np.array(["Down"] * 252)
labels[probs > 0.5] = "Up"
confusion_table(labels , D_test)

Truth,Down,Up
Predicted,,
Down,77,97
Up,34,44


In [19]:
np.mean(labels == D_test) , np.mean(labels != D_test)

(np.float64(0.4801587301587302), np.float64(0.5198412698412699))

In [20]:
X = MS(['Lag1' , 'Lag2']).fit_transform(Smarket)

X_train = X.loc[train]
y_train = y.loc[train]
X_test = X.loc[~train]
y_test = y.loc[~train]

glm_train = sm.GLM(y_train , X_train , family = sm.families.Binomial())
result = glm_train.fit()
summarize(result)
probs = result.predict(exog=X_test)

In [21]:
D = Smarket.Direction
D_train , D_test = D.loc[train] , D.loc[~train]

In [22]:
labels = np.array(["Down"] * 252)
labels[probs > 0.5] = "Up"
confusion_table(labels , D_test)

Truth,Down,Up
Predicted,,
Down,35,35
Up,76,106


In [23]:
np.mean(labels == D_test) , np.mean(labels != D_test)

(np.float64(0.5595238095238095), np.float64(0.44047619047619047))

In [24]:
X_test = pd.DataFrame({ 'Lag1' : [1.2 , 1.5],
                        'Lag2' : [1.1 , -0.8]})
model = MS(['Lag1' , 'Lag2']).fit(Smarket)
test_data = model.transform(X_test)
result.predict(test_data)


0    0.479146
1    0.496094
dtype: float64

In [25]:
lda = LDA(store_covariance=True)

In [26]:
train = Smarket.Year < 2005
X = MS(['Lag1' , 'Lag2']).fit_transform(Smarket)
X_train , X_test = X.loc[train] , X.loc[~train]

D = Smarket.Direction
D_train , D_test = D.loc[train] , D.loc[~train]

X_train , X_test = [M.drop(columns = ['intercept']) for M in [X_train , X_test]]
lda.fit(X_train , D_train)

,solver,'svd'
,shrinkage,None
,priors,None
,n_components,None
,store_covariance,True
,tol,0.0001
,covariance_estimator,None


In [27]:
lda.means_

array([[ 0.04279022,  0.03389409],
       [-0.03954635, -0.03132544]])

In [28]:
lda.classes_

array(['Down', 'Up'], dtype='<U4')

In [29]:
lda.priors_

array([0.49198397, 0.50801603])

In [30]:
lda.scalings_

array([[-0.64201904],
       [-0.51352928]])

In [31]:
lda_pred = lda.predict(X_test)
confusion_table(lda_pred , D_test)

Truth,Down,Up
Predicted,,
Down,35,35
Up,76,106


In [32]:
lda_prob = lda.predict_proba(X_test)
np.all(
    np.where(lda_prob[:,1] >= 0.5, 'Up','Down') == lda_pred
)

np.True_

In [33]:
np.all(
    [lda.classes_[i] for i in np.argmax(lda_prob, 1)] == lda_pred
)

np.True_

In [34]:
qda = QDA(store_covariance=True)
qda.fit(X_train, D_train)

,priors,None
,reg_param,0.0
,store_covariance,True
,tol,0.0001


In [35]:
qda.means_ , qda.priors_

(array([[ 0.04279022,  0.03389409],
        [-0.03954635, -0.03132544]]),
 array([0.49198397, 0.50801603]))

In [36]:
qda.covariance_[0]

array([[ 1.50662277, -0.03924806],
       [-0.03924806,  1.53559498]])

In [37]:
qda_pred = qda.predict(X_test)
confusion_table(qda_pred , D_test)

Truth,Down,Up
Predicted,,
Down,30,20
Up,81,121


In [38]:
np.mean(qda_pred == D_test)

np.float64(0.5992063492063492)

In [39]:
NB = GaussianNB()
NB.fit(X_train , D_train)

,priors,None
,var_smoothing,1e-09


In [40]:
NB.classes_

array(['Down', 'Up'], dtype='<U4')

In [41]:
NB.class_prior_

array([0.49198397, 0.50801603])

In [42]:
NB.theta_

array([[ 0.04279022,  0.03389409],
       [-0.03954635, -0.03132544]])

In [43]:
NB.var_

array([[1.50355429, 1.53246749],
       [1.51401364, 1.48732877]])

In [44]:
X_train[D_train == "Down"].var(ddof=0)

Lag1    1.503554
Lag2    1.532467
dtype: float64

In [45]:
nb_labels = NB.predict(X_test)
confusion_table(nb_labels , D_test)

Truth,Down,Up
Predicted,,
Down,29,20
Up,82,121


In [46]:
NB.predict_proba(X_test)[:5]

array([[0.4873288 , 0.5126712 ],
       [0.47623584, 0.52376416],
       [0.46529531, 0.53470469],
       [0.47484469, 0.52515531],
       [0.49020587, 0.50979413]])

In [47]:
knn1 = KNeighborsClassifier(n_neighbors=1)
knn1.fit(X_train , D_train)
knn1_pred = knn1.predict(X_test)
np.mean(knn1_pred == D_test)

np.float64(0.5)

In [48]:
confusion_table(knn1_pred , D_test)

Truth,Down,Up
Predicted,,
Down,43,58
Up,68,83


In [49]:
knn3 = KNeighborsClassifier(n_neighbors=3)
knn3.fit(X_train , D_train)
knn3_pred = knn3.predict(X_test)
np.mean(knn3_pred == D_test)

np.float64(0.5317460317460317)

In [50]:
confusion_table(knn3_pred , D_test)

Truth,Down,Up
Predicted,,
Down,48,55
Up,63,86


In [51]:
Caravan = load_data("Caravan")
Purchase = Caravan.Purchase
Purchase.value_counts()

Purchase
No     5474
Yes     348
Name: count, dtype: int64

In [52]:
feature_df = Caravan.drop(columns=['Purchase'])

In [53]:
scaler = StandardScaler(with_mean=True , with_std=True , copy=True)

In [54]:
scaler.fit(feature_df)

,copy,True
,with_mean,True
,with_std,True


In [55]:
X_std = scaler.transform(feature_df)

In [56]:
feature_std = pd.DataFrame(X_std , columns=[feature_df.columns])
feature_std.std()

MOSTYPE     1.000086
MAANTHUI    1.000086
MGEMOMV     1.000086
MGEMLEEF    1.000086
MOSHOOFD    1.000086
              ...   
AZEILPL     1.000086
APLEZIER    1.000086
AFIETS      1.000086
AINBOED     1.000086
ABYSTAND    1.000086
Length: 85, dtype: float64

In [57]:
X_train , X_test , y_train , y_test = train_test_split(feature_std , Purchase , test_size=1000 , random_state=0)

In [58]:
knn1 = KNeighborsClassifier(n_neighbors=1)
knn1.fit(X_train , y_train)
knn1_pred = knn1.predict(X_test)
np.mean(knn1_pred != y_test) , np.mean(y_test != "No")

(np.float64(0.111), np.float64(0.067))

In [60]:
confusion_table(knn1_pred , y_test)

Truth,No,Yes
Predicted,,
No,880,58
Yes,53,9


In [61]:
for K in range(1 , 6):
    knn = KNeighborsClassifier(n_neighbors=K)
    knn.fit(X_train , y_train)
    knn_pred = knn.predict(X_test)
    C = confusion_table(knn_pred , y_test)
    templ = ('K={0:d}: # predicted to rent: {1:>2},' +
            ' # who did rent {2:d}, accuracy {3:.1%}')

    rent_pred = C.loc['Yes'].sum()
    rent_real = C.loc['Yes' , 'Yes'].sum()
    accuracy = rent_real / rent_pred
    print(templ.format(K , rent_pred , rent_real , accuracy))

K=1: # predicted to rent: 62, # who did rent 9, accuracy 14.5%
K=2: # predicted to rent:  6, # who did rent 1, accuracy 16.7%
K=3: # predicted to rent: 20, # who did rent 3, accuracy 15.0%
K=4: # predicted to rent:  4, # who did rent 0, accuracy 0.0%
K=5: # predicted to rent:  7, # who did rent 1, accuracy 14.3%


In [62]:
logit = LogisticRegression(C = 1e10 , solver="liblinear")
logit.fit(X_train , y_train)
logit_pred = logit.predict_proba(X_test)
# print(logit_pred)
logit_labels = np.where(logit_pred[:,1] > 5 , "Yes" , "No")

confusion_table(logit_labels , y_test)

Truth,No,Yes
Predicted,,
No,933,67
Yes,0,0


In [63]:
logit_labels = np.where(logit_pred[:,1] > 0.25 , "Yes" , "No")

confusion_table(logit_labels , y_test)

Truth,No,Yes
Predicted,,
No,913,58
Yes,20,9


In [64]:
Bike = load_data("Bikeshare")

In [65]:
Bike.columns , Bike.shape

(Index(['season', 'mnth', 'day', 'hr', 'holiday', 'weekday', 'workingday',
        'weathersit', 'temp', 'atemp', 'hum', 'windspeed', 'casual',
        'registered', 'bikers'],
       dtype='object'),
 (8645, 15))

In [68]:
Bike

,season,mnth,day,hr,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,bikers
0,1,Jan,1,0,0,6,0,clear,0.24,0.2879,0.81,0.0000,3,13,16
1,1,Jan,1,1,0,6,0,clear,0.22,0.2727,0.80,0.0000,8,32,40
2,1,Jan,1,2,0,6,0,clear,0.22,0.2727,0.80,0.0000,5,27,32
3,1,Jan,1,3,0,6,0,clear,0.24,0.2879,0.75,0.0000,3,10,13
4,1,Jan,1,4,0,6,0,clear,0.24,0.2879,0.75,0.0000,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8640,1,Dec,365,19,0,6,0,clear,0.42,0.4242,0.54,0.2239,19,73,92
8641,1,Dec,365,20,0,6,0,clear,0.42,0.4242,0.54,0.2239,8,63,71
8642,1,Dec,365,21,0,6,0,clear,0.40,0.4091,0.58,0.1940,2,50,52
8643,1,Dec,365,22,0,6,0,clear,0.38,0.3939,0.62,0.1343,2,36,38


In [77]:
vars = ['mnth',
 'hr',
 'workingday',
 'temp',
 'weathersit']
X = MS(vars).fit_transform(Bike)
y = Bike["bikers"]
model = sm.OLS(y , X).fit()
summarize(model)

,coef,std err,t,P>|t|
intercept,-68.6317,5.307,-12.932,0.000
mnth[Feb],6.8452,4.287,1.597,0.110
mnth[March],16.5514,4.301,3.848,0.000
mnth[April],41.4249,4.972,8.331,0.000
mnth[May],72.5571,5.641,12.862,0.000
mnth[June],67.8187,6.544,10.364,0.000
mnth[July],45.3245,7.081,6.401,0.000
mnth[Aug],53.2430,6.640,8.019,0.000
mnth[Sept],66.6783,5.925,11.254,0.000
mnth[Oct],75.8343,4.950,15.319,0.000


In [ ]:
hr_encode = contrast('hr' , 'sum')
mnth_encode = contrast('hr' , 'sum')
vars = [mnth_encode,
 hr_encode,
 'workingday',
 'temp',
 'weathersit']
X2 = MS(vars).fit_transform(Bike)
model2 = sm.OLS(y , X2).fit()
summarize(model2)

,coef,std err,t,P>|t|
intercept,3.757280e+01,2.683000e+00,14.005,0.000
hr[0],-1.965000e+13,1.890000e+13,-1.038,0.299
hr[1],-3.285000e+13,1.850000e+13,-1.771,0.077
hr[2],5.447000e+12,4.230000e+12,1.287,0.198
hr[3],9.768000e+13,5.280000e+13,1.851,0.064
hr[4],1.609000e+14,1.080000e+14,1.488,0.137
hr[5],3.058000e+14,2.950000e+14,1.038,0.299
hr[6],1.795000e+14,3.270000e+14,0.549,0.583
hr[7],7.453000e+14,1.990000e+14,3.754,0.000
hr[8],1.342000e+14,2.770000e+14,0.485,0.627
